# Neighbour checks for quality control flags
Covers QC16-25

## Table of contents
[Utility neighbour functions](#utility-neighbour-functions)  
[QC16 Daily neighbours (wet)](#QC16---Daily-neighbours-wet)  
[QC17 Hourly neighbours (wet)](#QC17---Hourly-neighbours-wet)  
[QC18 Daily neighbours (dry)](#QC18---Daily-neighbours-dry)  
[QC19 Hourly neighbours (dry)](#QC19---Hourly-neighbours-dry)  
[QC20 Monthly neighbours](#QC20---Monthly-neighbours)  
[QC21 Timing offset](#QC21---Timing-offset)  
[QC22 Pre-QC Affinity](#QC22---Pre-QC-Affinity)  
[QC23 Pre-QC Pearson](#QC23---Pre-QC-Pearson)  
[QC24 Daily factor](#QC24---Daily-factor)  
[QC25 Monthly factor](#QC25---Monthly-factor)  

See '3.3.4 Neighbouring gauge checks on large values' in Lewis et al. (2021)

In [1]:
import datetime
import glob

import polars as pl
import numpy as np

import geopy.distance

In [2]:
TARGET_STATION_ID = "DE_02483"
DISTANCE_THRESHOLD = 50 # 50 km
OVERLAP_THRESHOLD = 365*3 # three years

## Data reading globals
GAUGE_DATA_PATH = "../data/gauge_data"
DATA_ROWS_TO_SKIP = 20 ## First 20 rows are metadata TODO: what if not?
UNIT_COL = "new_units" ## There is an original_units col too TODO: think of way to do this for different data

MULTIPLYING_FACTORS = {"hourly": 24, "daily": 1} ## compared to daily reference


In [3]:
def read_metadata(data_path):
    metadata = {}

    with open(data_path, 'r') as f:
        while True:
            key, val = f.readline().strip().split(':', maxsplit=1)
            key = key.lower().replace(' ', '_')
            metadata[key.strip()] = val.strip()
            if key == 'other':
                break
    return metadata

In [4]:
station_metadata = read_metadata(data_path='../data/gauge_data/DE_02483.txt')
station_metadata['start_datetime'] = datetime.datetime.strptime(station_metadata['start_datetime'], '%Y%m%d%H')
station_metadata['end_datetime'] = datetime.datetime.strptime(station_metadata['end_datetime'], '%Y%m%d%H')

In [5]:
def add_datetime_to_gauge_data(station_metadata, gauge_data, time_multiplying_factor):
    """
    Add datetime column to gauge data using metadata from that gauge.
    NOTE: Could maybe extend so can find metadata if not provided?
    """
    startdate = station_metadata['start_datetime']
    enddate = station_metadata['end_datetime']
    assert isinstance(startdate, datetime.datetime), "Please convert start_ and end_datetime to datetime.datetime objects"

    date_interval = []
    delta_days = (enddate+datetime.timedelta(days=1) - startdate).days
    for i in range(delta_days * time_multiplying_factor):
        date_interval.append(startdate + datetime.timedelta(hours=i))

    ## add datetime column
    assert len(gauge_data) == len(date_interval)
    gauge_data = gauge_data.with_columns(time=pl.Series(date_interval)) ## set time columns

    return gauge_data


In [6]:
rain_col = f'rain_{station_metadata['original_units']}'

In [7]:
## read in gauge data
target_gauge = pl.read_csv('../data/gauge_data/DE_02483.txt', skip_rows=20, schema_overrides={rain_col: pl.Float64})

In [8]:
target_gauge = add_datetime_to_gauge_data(station_metadata, target_gauge, time_multiplying_factor=MULTIPLYING_FACTORS['hourly'])
target_gauge = target_gauge.select(['time', rain_col]) ## Reorder (to look nice)

In [9]:
## make no data vals nans
target_gauge = target_gauge.with_columns(pl.when(pl.col(rain_col) == int(station_metadata['no_data_value'])).then(np.nan).otherwise(pl.col(rain_col)).alias(rain_col))

In [10]:
target_gauge.head()

time,rain_mm
datetime[μs],f64
2006-01-01 00:00:00,0.9
2006-01-01 01:00:00,0.3
2006-01-01 02:00:00,0.3
2006-01-01 03:00:00,0.0
2006-01-01 04:00:00,0.0


# Utility neighbour functions
TODO: convert to Classes

### Part 1. Make or read summary metadata of stations

In [11]:
## Could work by checking if metadata already exists (or user can input)
all_gauge_data_paths = glob.glob(f"{GAUGE_DATA_PATH}/*.txt")
all_gauge_data_paths

['../data/gauge_data/DE_06303.txt',
 '../data/gauge_data/DE_02718.txt',
 '../data/gauge_data/DE_04313.txt',
 '../data/gauge_data/DE_00390.txt',
 '../data/gauge_data/DE_02483.txt',
 '../data/gauge_data/DE_03215.txt',
 '../data/gauge_data/DE_00389.txt',
 '../data/gauge_data/DE_06264.txt',
 '../data/gauge_data/DE_01300.txt',
 '../data/gauge_data/DE_04488.txt',
 '../data/gauge_data/DE_00310.txt']

In [12]:
all_station_metadata_list = []
for ind, file in enumerate(all_gauge_data_paths):
    one_station_metadata = read_metadata(data_path=file)
    all_station_metadata_list.append(one_station_metadata)


In [13]:
all_station_metadata = pl.from_dicts(all_station_metadata_list)
all_station_metadata = all_station_metadata.with_columns(
    pl.col("latitude").cast(pl.Float64),
    pl.col("longitude").cast(pl.Float64),
    (pl.col("start_datetime")+'00').str.strptime(pl.Datetime, "%Y%m%d%H%M"),
    (pl.col("end_datetime")+'00').str.strptime(pl.Datetime, "%Y%m%d%H%M"),
)
all_station_metadata.head()

station_id,country,original_station_number,original_station_name,path_to_original_data,latitude,longitude,start_datetime,end_datetime,elevation,number_of_records,percent_missing_data,original_timestep,new_timestep,original_units,new_units,time_zone,daylight_saving_info,no_data_value,resolution,other
str,str,str,str,str,f64,f64,datetime[μs],datetime[μs],str,str,str,str,str,str,str,str,str,str,str,str
"""DE_06303""","""Germany""","""06303""","""NA""","""B:/INTENSE data/Original data/…",51.2884,8.5907,2006-01-01 00:00:00,2010-12-31 23:00:00,"""588m""","""43824""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""
"""DE_02718""","""Germany""","""02718""","""NA""","""B:/INTENSE data/Original data/…",51.288,8.7928,2006-01-01 00:00:00,2010-12-31 23:00:00,"""458m""","""43824""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""
"""DE_04313""","""Germany""","""04313""","""NA""","""B:/INTENSE data/Original data/…",51.4966,8.4342,2006-01-01 00:00:00,2010-12-31 23:00:00,"""361m""","""43824""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""
"""DE_00390""","""Germany""","""00390""","""NA""","""B:/INTENSE data/Original data/…",50.9837,8.3679,2006-01-01 00:00:00,2010-12-31 23:00:00,"""610m""","""43824""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""
"""DE_02483""","""Germany""","""02483""","""NA""","""B:/INTENSE data/Original data/…",51.1803,8.4891,2006-01-01 00:00:00,2010-12-31 23:00:00,"""839m""","""43824""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""


### Part 2. Compute distance from target station

In [14]:
def compute_distance_from_target_id(metadata, target_id):
    target_station = metadata.filter(pl.col("station_id") == target_id)
    target_latlon = (target_station['latitude'].item(), target_station['longitude'].item())

    neighbour_distances = {}
    for other_station_id, other_lat, other_lon in metadata[['station_id', 'latitude', 'longitude']].rows():

        neighbour_distances[other_station_id] = geopy.distance.geodesic(target_latlon, (other_lat, other_lon)).kilometers
    return neighbour_distances

In [15]:
neighbours_distances = compute_distance_from_target_id(all_station_metadata, TARGET_STATION_ID)
neighbours_distances

{'DE_06303': 13.96385857171046,
 'DE_02718': 24.361752991036987,
 'DE_04313': 35.397225659812456,
 'DE_00390': 23.462778369145934,
 'DE_02483': 0.0,
 'DE_03215': 15.710073576478786,
 'DE_00389': 18.844342416100808,
 'DE_06264': 28.343769567230837,
 'DE_01300': 24.64263807685722,
 'DE_04488': 15.929311137997068,
 'DE_00310': 13.134594190689885}

In [16]:
# ALTERNATIVE: do we want to avoid using geopy.distance and simply write a distance function?
# # ALTERNATIVE: maybe precomupte a matrix of distances?
# all_station_dist_mtx = scipy.spatial.distance.cdist(all_station_metadata[['latitude', 'longitude']].rows(),
#                                         all_station_metadata[['latitude', 'longitude']].rows(),
#                                         metric=lambda pnt1, pnt2: geopy.distance.geodesic(pnt1, pnt2).kilometers)

In [17]:
neighbours_distances_df = pl.DataFrame({"station_id": neighbours_distances.keys(),
              "distances": neighbours_distances.values()
              })
neighbours_distances_df

station_id,distances
str,f64
"""DE_06303""",13.963859
"""DE_02718""",24.361753
"""DE_04313""",35.397226
"""DE_00390""",23.462778
"""DE_02483""",0.0
…,…
"""DE_00389""",18.844342
"""DE_06264""",28.34377
"""DE_01300""",24.642638


In [18]:
## Subset based on 50 km
close_neighbours = neighbours_distances_df.filter(
    (pl.col("distances") <= DISTANCE_THRESHOLD) &
    (pl.col("distances") != 0)
    )

## closest 10 
close_neighbours.sort('distances')[:10]

station_id,distances
str,f64
"""DE_00310""",13.134594
"""DE_06303""",13.963859
"""DE_03215""",15.710074
"""DE_04488""",15.929311
"""DE_00389""",18.844342
"""DE_00390""",23.462778
"""DE_02718""",24.361753
"""DE_01300""",24.642638
"""DE_06264""",28.34377


### Part 3. Compute the temporal overlap

In [19]:
def compute_overlap_days(start_1, end_1, start_2, end_2):
    ## TODO: add cast to datetime functionality/checks
    ## compute overlap
    overlap_start = max(start_1, start_2)
    overlap_end = min(end_1, end_2)

    overlap_days = max(0, (overlap_end - overlap_start).days)

    return overlap_days

In [20]:
def compute_overlap_days_from_target_id(metadata, target_id):
    target_station = metadata.filter(pl.col("station_id") == target_id)
    start_1, end_1 = target_station['start_datetime'].item(), target_station['end_datetime'].item()

    neighbour_overlap_days = {}
    for other_station_id, start_2, end_2 in metadata[['station_id', 'start_datetime', 'end_datetime']].rows():
        if target_id == other_station_id:
            continue

        neighbour_overlap_days[other_station_id] = compute_overlap_days(start_1, end_1, start_2, end_2)
    return neighbour_overlap_days

In [21]:
neighbour_overlap_days = compute_overlap_days_from_target_id(all_station_metadata, TARGET_STATION_ID)
neighbour_overlap_days

{'DE_06303': 1825,
 'DE_02718': 1825,
 'DE_04313': 1825,
 'DE_00390': 1825,
 'DE_03215': 1309,
 'DE_00389': 425,
 'DE_06264': 1825,
 'DE_01300': 1825,
 'DE_04488': 1613,
 'DE_00310': 1825}

#### Subset based on 3 years

In [22]:
neighbour_overlap_days_df = pl.DataFrame({"station_id": neighbour_overlap_days.keys(),
              "overlap_days": neighbour_overlap_days.values()
              })
neighbour_overlap_days_df

station_id,overlap_days
str,i64
"""DE_06303""",1825
"""DE_02718""",1825
"""DE_04313""",1825
"""DE_00390""",1825
"""DE_03215""",1309
"""DE_00389""",425
"""DE_06264""",1825
"""DE_01300""",1825
"""DE_04488""",1613


In [23]:
neighbour_overlap_days_df.filter(
    pl.col("overlap_days") >= OVERLAP_THRESHOLD
)

station_id,overlap_days
str,i64
"""DE_06303""",1825
"""DE_02718""",1825
"""DE_04313""",1825
"""DE_00390""",1825
"""DE_03215""",1309
"""DE_06264""",1825
"""DE_01300""",1825
"""DE_04488""",1613
"""DE_00310""",1825


## Part 4. Bring together to get neighbours both close and overlapping

In [24]:
num_closest_gauges = 10 ## based on IntenseQC

In [25]:
## Subset based on 50 km
close_neighbour_ids = neighbours_distances_df.filter(
    (pl.col("distances") <= DISTANCE_THRESHOLD) &
    (pl.col("distances") != 0)
)
## closest 10 values
closest_neighbour_ids = close_neighbour_ids.sort('distances')[:num_closest_gauges]['station_id'].to_list()

## Subset based on 3 years
overlapping_neighbour_ids = neighbour_overlap_days_df.filter(
    pl.col("overlap_days") >= OVERLAP_THRESHOLD
)['station_id'].to_list()

In [26]:
all_neighbour_ids = set(overlapping_neighbour_ids).intersection(set(closest_neighbour_ids))
all_neighbour_ids

{'DE_00310',
 'DE_00390',
 'DE_01300',
 'DE_02718',
 'DE_03215',
 'DE_04313',
 'DE_04488',
 'DE_06264',
 'DE_06303'}

In [27]:
all_neighbour_ids_paths = {}
for id in all_neighbour_ids:
    ids_path = glob.glob(f"{GAUGE_DATA_PATH}/*{id}.txt")
    assert len(ids_path) == 1, f"There are {len(ids_path)} data files for {id}"
    all_neighbour_ids_paths[id] = ids_path[0]

In [28]:
all_neighbour_ids_paths

{'DE_03215': '../data/gauge_data/DE_03215.txt',
 'DE_02718': '../data/gauge_data/DE_02718.txt',
 'DE_06303': '../data/gauge_data/DE_06303.txt',
 'DE_01300': '../data/gauge_data/DE_01300.txt',
 'DE_00310': '../data/gauge_data/DE_00310.txt',
 'DE_04313': '../data/gauge_data/DE_04313.txt',
 'DE_06264': '../data/gauge_data/DE_06264.txt',
 'DE_04488': '../data/gauge_data/DE_04488.txt',
 'DE_00390': '../data/gauge_data/DE_00390.txt'}

## Part 6. Get neighbouring GDSR gauge by ID (an example)

In [29]:
def get_neighbour_gauge_data(neighbour_gauge_id, time_multiplying_factor):
    data_path = all_neighbour_ids_paths[neighbour_gauge_id]
    station_metadata = all_station_metadata.filter(pl.col("station_id") == neighbour_gauge_id)
    assert len(station_metadata) == 1, f"There are {len(station_metadata)} metadata values for {neighbour_gauge_id}. Investigate because there should only be one"
    station_metadata = station_metadata.to_dicts()[0] ## convert df to a dict

    ## Read in gauge data
    units = station_metadata[UNIT_COL]
    rain_col = f'rain_{units}'
    gauge_data = pl.read_csv(data_path, skip_rows=DATA_ROWS_TO_SKIP, schema_overrides={rain_col: pl.Float64})

    ## make datetime column
    gauge_data_w_dates = add_datetime_to_gauge_data(station_metadata, gauge_data, time_multiplying_factor=time_multiplying_factor)
    gauge_data_w_dates = gauge_data_w_dates.select(['time', rain_col]) ## Reorder (to look nice)

    return gauge_data_w_dates


In [30]:
get_neighbour_gauge_data(neighbour_gauge_id='DE_06264', time_multiplying_factor=MULTIPLYING_FACTORS['hourly'])

time,rain_mm
datetime[μs],f64
2006-01-01 00:00:00,0.0
2006-01-01 01:00:00,0.1
2006-01-01 02:00:00,0.0
2006-01-01 03:00:00,0.0
2006-01-01 04:00:00,0.0
…,…
2010-12-31 19:00:00,0.0
2010-12-31 20:00:00,0.0
2010-12-31 21:00:00,0.0


## Part 7. Get neighbouring GPCC gauge by ID (an example)

#### Note:
In the original methodology, GPCC is extracted on the fly

Hence, this needs to be refactored

In [31]:
import zipfile
import pandas as pd


In [32]:
existing_gpcc_daily_paths = {}
existing_gpcc_monthly_paths = {}
for neighbour_id in all_neighbour_ids_paths.keys():
    gpcc_id = neighbour_id.split('DE_')[1].lstrip("0")
    existing_gpcc_daily_paths[neighbour_id] = glob.glob(f"../data/GPCC/tw_{gpcc_id}.zip")
    existing_gpcc_monthly_paths[neighbour_id] = glob.glob(f"../data/GPCC/mw_{gpcc_id}.zip")

In [33]:
existing_gpcc_daily_paths

{'DE_03215': ['../data/GPCC/tw_3215.zip'],
 'DE_02718': [],
 'DE_06303': ['../data/GPCC/tw_6303.zip'],
 'DE_01300': [],
 'DE_00310': ['../data/GPCC/tw_310.zip'],
 'DE_04313': [],
 'DE_06264': [],
 'DE_04488': [],
 'DE_00390': []}

In [34]:
example_dat_path = existing_gpcc_daily_paths['DE_06303'][0]
f = zipfile.ZipFile(example_dat_path).open("tw_6303.dat")
example_dat = pl.from_pandas(pd.read_csv(f, skiprows=1, header=None, sep=r'\s+'))

example_dat

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,12,2002,3.0,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
2,12,2002,0.1,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
3,12,2002,0.2,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
4,12,2002,0.1,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
5,12,2002,1.0,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
27,12,2018,0.0,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
28,12,2018,1.0,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
29,12,2018,10.5,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9


# QC16 - Daily neighbours (wet)
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- Many, although mainly this implementation opens the neighbour wet and dry functions to parameter tweaking

In [35]:
## resample data into daily (NOTE: OG method makes sure this is 7am-7pm)


In [36]:
example_id = 'DE_06303'

In [37]:
target_gauge

time,rain_mm
datetime[μs],f64
2006-01-01 00:00:00,0.9
2006-01-01 01:00:00,0.3
2006-01-01 02:00:00,0.3
2006-01-01 03:00:00,0.0
2006-01-01 04:00:00,0.0
…,…
2010-12-31 19:00:00,0.0
2010-12-31 20:00:00,0.0
2010-12-31 21:00:00,0.0


# QC17 - Hourly neighbours (wet) 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

In [38]:
example_id = 'DE_06264'

In [39]:
neighbouring_gauge = get_neighbour_gauge_data(neighbour_gauge_id=example_id, time_multiplying_factor=MULTIPLYING_FACTORS['hourly'])

In [40]:
neighbouring_gauge

time,rain_mm
datetime[μs],f64
2006-01-01 00:00:00,0.0
2006-01-01 01:00:00,0.1
2006-01-01 02:00:00,0.0
2006-01-01 03:00:00,0.0
2006-01-01 04:00:00,0.0
…,…
2010-12-31 19:00:00,0.0
2010-12-31 20:00:00,0.0
2010-12-31 21:00:00,0.0


In [61]:
for id in all_neighbour_ids_paths.keys():
    print(id)
    one_neighbouring_gauge = get_neighbour_gauge_data(neighbour_gauge_id=id, time_multiplying_factor=MULTIPLYING_FACTORS['hourly'])
    joined_gauges = target_gauge.join(one_neighbouring_gauge, on='time', suffix=f'_{id}')
    joined_gauges = joined_gauges.drop_nans()

    ## NOTE: is this necessary? Why not just read resolution of each data?
    a = np.around(joined_gauges.filter(pl.col("rain_mm") >= 0.1).min()['rain_mm'], 1)[0]
    b = np.around(joined_gauges.filter(pl.col(f"rain_mm_{id}") >= 0.1).min()[f"rain_mm_{id}"], 1)[0]
    p = max(a, b, 0.1)
    print(a, b, p)

    ## TODO: rename all variables
    joined_gauges_duplicates = joined_gauges.with_columns(
        pl.when(
            (pl.col('rain_mm') > p) &
            (pl.col(f"rain_mm_{id}") > p)
        ).then(1)
        .when(
            (pl.col('rain_mm') == p) &
            (pl.col(f"rain_mm_{id}") == p),
        ).then(1)
        .when(
            (pl.col('rain_mm') == p) &
            (pl.col(f"rain_mm_{id}") > p),
        ).then(0)
        .when(
            (pl.col('rain_mm') > p) &
            (pl.col(f"rain_mm_{id}") == p)
        ).then(0)
        .otherwise(np.nan)
        .alias("duplicate")
    )

    joined_gauges = joined_gauges.with_columns(
        pl.when(
            (pl.col("rain_mm") > 0) & (pl.col(f"rain_mm_{id}") > 0)
        ).then(pl.col("rain_mm") / pl.col(f"rain_mm_{id}"))
        .otherwise(np.nan)
        .alias("factor")
    )

    match = joined_gauges_duplicates['duplicate'].value_counts().filter(pl.col('duplicate') == 1)['count'].item()
    diff = joined_gauges_duplicates['duplicate'].value_counts().filter(pl.col('duplicate') == 0)['count'].item()
    perc = match / (match + diff)
    p_corr = np.corrcoef(joined_gauges['rain_mm'], joined_gauges[f"rain_mm_{id}"])[0, 1]
    f_mean = joined_gauges['factor'].drop_nans().mean()
    print(f"diff: {diff}, match:{match}")
    print(f"perc: {perc}, p_corr: {p_corr}, f_mean: {f_mean}")

    print("#"*15)

DE_03215
0.1 0.1 0.1
diff: 786, match:2546
perc: 0.7641056422569028, p_corr: 0.012027849830275862, f_mean: 3.4620312158992337
###############
DE_02718
0.1 0.1 0.1
diff: 1103, match:3559
perc: 0.7634062634062634, p_corr: 0.001478646393856345, f_mean: 4.040384692933253
###############
DE_06303
0.1 0.1 0.1
diff: 1064, match:4668
perc: 0.8143754361479414, p_corr: 0.0015406217259350391, f_mean: 2.7597544248988704
###############
DE_01300
0.1 0.1 0.1
diff: 1067, match:4426
perc: 0.8057527762606954, p_corr: 0.0013590409692924373, f_mean: 3.3140483159354046
###############
DE_00310
0.1 0.1 0.1
diff: 1062, match:4482
perc: 0.8084415584415584, p_corr: 0.002986867106623167, f_mean: 2.9478909851492725
###############
DE_04313
0.1 0.1 0.1
diff: 1004, match:3273
perc: 0.7652560205751695, p_corr: 0.00020607728354890315, f_mean: 4.1753082711347425
###############
DE_06264
0.1 0.1 0.1
diff: 1016, match:3588
perc: 0.7793223284100782, p_corr: 0.0014740098747488808, f_mean: 3.8051933291498767
############

# QC18 - Daily neighbours (dry) 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC19 - Hourly neighbours (dry) 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC20 - Monthly neighbours
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC21 - Timing offset 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC22 - Pre-QC Affinity  
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC23 - Pre-QC Pearson
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC24 - Daily factor  
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC25 - Monthly factor
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 